In [1]:
import wandb

In [12]:
wandb.login()

True

In [13]:
?wandb.login

Signature:
wandb.login(
    anonymous: Optional[Literal['must', 'allow', 'never']] = None,
    key: Optional[str] = None,
    relogin: Optional[bool] = None,
    host: Optional[str] = None,
    force: Optional[bool] = None,
    timeout: Optional[int] = None,
    verify: bool = False,
) -> bool
Docstring:
Set up W&B login credentials.

By default, this will only store credentials locally without
verifying them with the W&B server. To verify credentials, pass
`verify=True`.

Args:
    anonymous: (string, optional) Can be "must", "allow", or "never".
        If set to "must", always log a user in anonymously. If set to
        "allow", only create an anonymous user if the user
        isn't already logged in. If set to "never", never log a
        user anonymously. Default set to "never".
    key: (string, optional) The API key to use.
    relogin: (bool, optional) If true, will re-prompt for API key.
    host: (string, optional) The host to connect to.
    force: (bool, optional) If true

In [22]:
sweep_config = {
    'method': 'bayes'
}

metric = {
    'name': 'loss',
    'goal': 'minimize'
}

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
    },
    'fc_layer_size': {
        'values': [128, 256, 512]
    },
    'dropout': {
        'values': [0.3, 0.4, 0.5]
    },
}

parameters_dict.update({
    'epochs': {
        'value': 1}
    })

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

sweep_config['parameters'] = parameters_dict

In [23]:
import pprint
pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [24]:
sweep_id = wandb.sweep(sweep_config, entity='lennie', project="wandb-pytorch-sweeps-demo")

Create sweep with ID: 0p7tk7a5
Sweep URL: https://wandb.ai/lennie/wandb-pytorch-sweeps-demo/sweeps/0p7tk7a5


In [25]:
sweep_id

'0p7tk7a5'

# ML code

In [26]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(project='wandb_demo',
                    config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})           

In [27]:
def build_dataset(batch_size):
   
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST(".", train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)

    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

# 4. Activate sweep agents

In [29]:
wandb.agent(sweep_id, train, count=15)

wandb: Agent Starting Run: ba1757qh with config:
wandb: 	batch_size: 72
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.013303348710964004
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


batch loss,██▄▃▃▃▂▂▂▂▂▁▂▃▂▂▂▃▂▂▁▁▂▂▁▁▂▁▁▂▃▁▁▁▁▂▂▁▁▁
epoch,▁
loss,▁
batch loss,0.35597
epoch,0
loss,0.52782


wandb: Agent Starting Run: lyae4yar with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.07321790164689267
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


batch loss,▄▂▂▃▃▃▅█▂▆▅▃▃▃▃▂▂▁▃▇▃▂▃▁▃▂▂▂▂▅▄▃▂▂▁▃▄▁▂▂
epoch,▁
loss,▁
batch loss,0.9902
epoch,0
loss,1.45464


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: e0k0oqa7 with config:
wandb: 	batch_size: 48
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.010493615548990289
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

batch loss,██▇▄▅▄▃▃▃▃▂▃▃▂▃▄▃▃▁▂▂▂▂▃▄▃▃▁▅▂▂▃▂▁▂▃▂▂▂▁
epoch,▁
loss,▁
batch loss,0.55605
epoch,0
loss,0.50113


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ijmtydoe with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.031565926467111816
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


batch loss,▆▃▅▂▃▄▃▄█▂▆▅▃▂▂▃▄▃▃▄▄▄▂▂▂▁▂▁▄▄▂▂▃▂▄▃▂▁▁▂
epoch,▁
loss,▁
batch loss,0.7081
epoch,0
loss,0.52393


wandb: Agent Starting Run: drj2azcy with config:
wandb: 	batch_size: 96
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.08554410132943008
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


batch loss,█▆▃▃▃▃▂▁▄▂▃▂▃▂▁▂▂▂▂▂▁▁▂▂▂▂▂▂▁▁▁▂▁▁▁▂▁▂▁▁
epoch,▁
loss,▁
batch loss,0.33334
epoch,0
loss,0.53849
